In [ ]:
import os
from db.models import FilingTag, FileLocation, File
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# create spreadsheet from df_results
df_results.to_csv('filing_tags_file_locations_summary.csv', index=False)

In [ ]:
import os
import psycopg
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve database credentials from environment variables
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")

# Establish a connection to the database
conn = psycopg.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST
)

# Create a cursor to execute SQL commands
cur = conn.cursor()

# SQL statements to create tables
sql_commands = [
    """
    CREATE TABLE prototype_runs (
      run_id SERIAL PRIMARY KEY,
      model_name TEXT NOT NULL,
      model_version TEXT NOT NULL,
      algorithm TEXT NOT NULL,
      hyperparams JSONB,
      tag_filter TEXT,
      created_at TIMESTAMPTZ DEFAULT now()
    );
    """,
    """
    CREATE TABLE prototype_members (
      run_id INTEGER REFERENCES prototype_runs(run_id) ON DELETE CASCADE,
      tag TEXT REFERENCES filing_tags(label),
      prototype_id SMALLINT DEFAULT 0,
      file_id INTEGER REFERENCES files(id),
      PRIMARY KEY (run_id, tag, prototype_id, file_id)
    );
    """,
    """
    ALTER TABLE tag_prototypes ADD COLUMN run_id INTEGER
      REFERENCES prototype_runs(run_id) ON DELETE SET NULL;
    """,
    """
    CREATE TABLE prototype_run_metrics (
      run_id INTEGER REFERENCES prototype_runs(run_id) ON DELETE CASCADE,
      metric_name TEXT,
      value NUMERIC,
      split TEXT,
      PRIMARY KEY (run_id, metric_name, split)
    );
    """
]

# Execute each command
for command in sql_commands:
    cur.execute(command)

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

In [ ]:
import os
from pathlib import Path
from itertools import islice


def visualize_directory_tree(
    dir_path: Path,
    level: int = -1,
    limit_to_directories: bool = False,
    length_limit: int = 1000000,
    exclusion_list: list = None
):
    """Given a directory Path object print a visual tree structure, with optional exclusions."""
    space = '    '
    branch = '│   '
    # pointers:
    tee = '├── '
    last = '└── '

    dir_path = Path(dir_path)  # accept string coerceable to Path
    files = 0
    directories = 0
    exclusion_set = set(exclusion_list) if exclusion_list else set()

    def inner(dir_path: Path, prefix: str = '', level=-1):
        nonlocal files, directories
        if not level:
            return  # 0, stop iterating
        if limit_to_directories:
            contents = [d for d in dir_path.iterdir() if d.is_dir() and d.name not in exclusion_set]
        else:
            contents = [d for d in dir_path.iterdir() if d.name not in exclusion_set]
        pointers = [tee] * (len(contents) - 1) + [last] if contents else []
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space
                yield from inner(path, prefix=prefix + extension, level=level - 1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1

    print(dir_path.name)
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        print(line)
    if next(iterator, None):
        print(f'... length_limit, {length_limit}, reached, counted:')
    print(f'\n{directories} directories' + (f', {files} files' if files else ''))
# exclude irrelevant directories
exclude_dirs = [
    '.venv',
    '__pycache__',
    'dev\__pycache__',
    '.git',
    'test_files'
]

visualize_directory_tree(
    dir_path= os.getcwd(),
    exclusion_list= exclude_dirs
)

In [ ]:
import os
from text_extraction.pdf_extraction import PDFTextExtractor

def test_pdf_text_extraction(directory_path):
    """
    Test PDF text extraction on all files in the given directory.
    
    Args:
        directory_path (str): Path to the directory containing PDF files.
    """
    extractor = PDFTextExtractor()
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = os.path.join(root, file)
                try:
                    print(f"Extracting text from: {file_path}")
                    text = extractor(file_path)
                    print(f"Extracted text (first 500 characters):\n{text[:500]}\n")
                except Exception as e:
                    print(f"Failed to extract text from {file_path}: {e}")

# Example usage
test_directory = "path/to/your/pdf/directory"
test_pdf_text_extraction(test_directory)

In [ ]:
import os
import shutil
from db.models import FileLocation, File, get_db_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func, or_

In [ ]:
import logging
import os
import shutil
import traceback 
import tempfile
from text_extraction.pdf_extraction import PDFTextExtractor
from text_extraction.basic_extraction import TextFileTextExtractor, get_extractor_for_file
from text_extraction.image_extraction import ImageTextExtractor
from text_extraction.office_doc_extraction import PresentationTextExtractor, SpreadsheetTextExtractor, WordFileTextExtractor
from text_extraction.web_extraction import HtmlTextExtractor, EmailTextExtractor
from text_extraction.extraction_utils import common_char_replacements, strip_diacritics, normalize_unicode
from logging_setups import setup_logger

setup_logger(name="NotebookLogger", notebook=True, level=logging.DEBUG)

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Initialize extractors for different file types
pdf_extractor = PDFTextExtractor()
txt_extractor = TextFileTextExtractor()
image_extractor = ImageTextExtractor()
presentation_extractor = PresentationTextExtractor()
spreadsheet_extractor = SpreadsheetTextExtractor()
word_extractor = WordFileTextExtractor()
html_extractor = HtmlTextExtractor()
email_extractor = EmailTextExtractor()

extractors_list = [
    pdf_extractor,
    txt_extractor,
    image_extractor,
    presentation_extractor,
    spreadsheet_extractor,
    word_extractor,
    html_extractor,
    email_extractor
]
display_text_len = 800
test_extraction_path = os.path.join(os.getcwd(), "test_files")

for file in os.listdir(test_extraction_path):
    file_path = os.path.join(test_extraction_path, file)
    def text_assessment(extracted_text):
        if not extracted_text:
            print(f"No text extracted from {file} in temporary directory")
        elif len(extracted_text) > display_text_len:
            print(f"Extracted text (first {display_text_len} characters) from {file}:\n{extracted_text[:display_text_len]}\n")
        else:
            print(f"Extracted text from {file}:\n{extracted_text}\n")

    if not os.path.isfile(file_path):
        print(f"Skipping {file}, not a file.")
        continue
    
    extractor = get_extractor_for_file(file_path=file_path, extractors=extractors_list)
    if extractor:
        try:
            print(f"Extracting text from {file} using {extractor.__class__.__name__}")
            text = extractor(file_path)
            text_assessment(text)
        
        except PermissionError as pe:
            # Handle PermissionError by copying the file to a temporary directory
            temp_dir = tempfile.gettempdir()
            temp_file_path = os.path.join(temp_dir, file)
            shutil.copy2(file_path, temp_file_path)
            print(f"Copied {file} to temporary directory: {temp_file_path}")
            
            # Retry extraction from the temporary file
            try:
                text = extractor(temp_file_path)
                text = common_char_replacements(text)
                text = strip_diacritics(text)
                text = normalize_unicode(text)
                text_assessment(text)

            except Exception as e:
                raise Exception(f"Failed to extract text from {file} in temporary directory: {e}")
        
        except Exception as e:

            print(f"Failed to extract text from {file}: {e}")
            traceback.print_exc()
    else:
        print(f"No suitable extractor found for {file}")


In [ ]:
# this cell is for removing erronious tag labels from the database
from sqlalchemy.orm import sessionmaker
from db_models import FileTagLabel, get_db_engine

def delete_file_tag_labels_by_hash(session, file_hash: str) -> int:
    """
    Delete FileTagLabel entries associated with the given file hash.
    Returns the number of rows deleted.
    """
    deleted_count = (
        session.query(FileTagLabel)
               .filter(FileTagLabel.file_hash == file_hash)
               .delete(synchronize_session=False)
    )
    session.commit()
    return deleted_count

hsh_lst = [
    "12e09752a751819a10d2357cbd72cc5104b06f8c",
    "9055a3c243f23f27ea82cc10046db2ed9aba401d",
    "6007b740d71580b20ae7a4d6b655af7b26a015be",
    "a0a13159ecdf28f465f330a21c0f4dad247def2b",
    "c8b1012ca42876d3a21f6dad2ca3c65bb325708a"
]


with sessionmaker(bind=get_db_engine())() as session:
    for hsh in hsh_lst:
        deleted_count = delete_file_tag_labels_by_hash(session, hsh)
        print(f"Deleted {deleted_count} FileTagLabel entries for hash {hsh}.")

In [1]:
from sqlalchemy.orm import sessionmaker
from db_models import FileEmbedding, get_db_engine

def delete_file_embeddings_by_hash(session, file_hash: str) -> int:
    """
    Delete FileEmbedding entries associated with the given file hash.
    Returns the number of rows deleted.
    """
    deleted_count = (
        session.query(FileEmbedding)
               .filter(FileEmbedding.file_hash == file_hash)
               .delete(synchronize_session=False)
    )
    session.commit()
    return deleted_count


hsh_lst = [
    "12e09752a751819a10d2357cbd72cc5104b06f8c",
    "9055a3c243f23f27ea82cc10046db2ed9aba401d",
    "6007b740d71580b20ae7a4d6b655af7b26a015be",
    "a0a13159ecdf28f465f330a21c0f4dad247def2b",
    "c8b1012ca42876d3a21f6dad2ca3c65bb325708a"
]

with sessionmaker(bind=get_db_engine())() as session:
    for hsh in hsh_lst:
        deleted_embeddings = delete_file_embeddings_by_hash(session, hsh)
        print(f"Deleted {deleted_embeddings} FileEmbedding entries for hash {hsh}.")



Deleted 1 FileEmbedding entries for hash 12e09752a751819a10d2357cbd72cc5104b06f8c.
Deleted 1 FileEmbedding entries for hash 9055a3c243f23f27ea82cc10046db2ed9aba401d.
Deleted 1 FileEmbedding entries for hash 6007b740d71580b20ae7a4d6b655af7b26a015be.
Deleted 1 FileEmbedding entries for hash a0a13159ecdf28f465f330a21c0f4dad247def2b.
Deleted 1 FileEmbedding entries for hash c8b1012ca42876d3a21f6dad2ca3c65bb325708a.


In [6]:
from db_models import FileEmbedding, FileLocation, File
from utils import extract_server_dirs
from db_models import get_db_engine
from sqlalchemy.orm import sessionmaker

SERVER_MOUNT_LOCATION = r"N:\PPDO\Records"

def find_embeddings_by_text_and_directory(session, search_str: str, dir_prefix: str):
    """
    Return all FileEmbedding rows whose source_text contains `search_str`
    and which have at least one FileLocation whose file_server_directories starts with `dir_prefix`.
    """

    q = (
        session.query(FileEmbedding)
               .join(File, FileEmbedding.file)                   # link to File
               .join(FileLocation, File.locations)               # link to FileLocation
               .filter(
                   FileEmbedding.source_text.ilike(f"%{search_str}%"),    # case-insensitive match
                   FileLocation.file_server_directories.like(f"{dir_prefix}%")
               )
               .distinct()
    )
    return q.all()

search_dirs = extract_server_dirs(full_path=r"N:\PPDO\Records\37xx   Oakes College\3701\3701", base_mount=SERVER_MOUNT_LOCATION)


with sessionmaker(bind=get_db_engine())() as session:

    # Example usage:
    matches = find_embeddings_by_text_and_directory(
        session=session,
        search_str="lumber",
        dir_prefix= search_dirs
    )

    for emb in matches:
        # find the matching location
        loc = next(
            loc for loc in emb.file.locations
            if loc.file_server_directories.startswith(search_dirs)
        )
        print(f"{loc.file_server_directories}\\{loc.filename} → {emb.source_text[:200]!r}")

37xx   Oakes College/3701/3701/F - Bid Documents and Contract Award/F8 - Contract\File 3701. F8. Contract Documents-College 7 (Oakes)-Book 1 of 2-August 1974.pdf → '3 Ve, ch a Grant No. 4-9-00668-0 CONSTRUCTION DOCUMENTS COLLEGE 7 UNIVERSITY OF CALIFORNIA, SANTA CRUZ SANTA CRUZ, CALIFORNIA 95064 Tomsick, C-3688 Architect McCue Boone Tomsick 631 Clay Street San Fr'
37xx   Oakes College/3701/3701/G - Construction/G5 - Inspectors' Daily Reports and Inspection Requests\3701.G5.oaks correspondence university inspection request.pdf → ',. TO * R EFERErfC?:. il T E j."\') i, l r-noM NeHark E I l *1 i . I l San Jose K. Bednar ov ILJ. Rush L------ UCSC - Santa Cruz - Oakes College. OAT~ 8/11/77 CC: As indicated in earlier correspondence'
37xx   Oakes College/3701/3701/G - Construction/G15 - Change Orders and Contract Amendments\3701.G15.ChangeOrder27-R.pdf → 'so ; ; TM rN Change Order Squib . Co December 11, 1975 date F. L. FACKLER Project College 7 (Oakes College) Residential Enclosed is Change